# Credit card fraud detection (supervised learning)

In [60]:
#importing libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble

In [61]:
df=pd.read_csv('credict_card_Supervised.csv')

In [3]:
df

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,1/1/2019 0:00,"""Heller, Gutmann and Zieme""",grocery_pos,107.23,Orient,WA,48.8878,-118.2105,149,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,49.159047,-118.186462,1
1,1/1/2019 0:00,Lind-Buckridge,entertainment,220.11,Malad City,ID,42.1808,-112.2620,4154,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,43.150704,-112.154481,1
2,1/1/2019 0:07,Kiehn Inc,grocery_pos,96.29,Grenada,CA,41.6125,-122.5258,589,Systems analyst,12/21/1945,413636e759663f264aae1819a4d4f231,41.657520,-122.230347,1
3,1/1/2019 0:09,Beier-Hyatt,shopping_pos,7.77,High Rolls Mountain Park,NM,32.9396,-105.8189,899,Naval architect,8/30/1967,8a6293af5ed278dea14448ded2685fea,32.863258,-106.520205,1
4,1/1/2019 0:21,Bruen-Yost,misc_pos,6.85,Freedom,WY,43.0172,-111.0292,471,"""Education officer, museum""",8/2/1967,f3c43d336e92a44fc2fb67058d5949e3,43.753735,-111.454923,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1/1/2019 6:20,Hudson-Ratke,grocery_pos,199.55,Littleton,CO,39.5994,-105.0044,320420,Water engineer,7/31/1975,a5c13edd756f36c026630d13672974bd,38.619691,-105.498059,1
96,1/1/2019 6:24,Harber Inc,gas_transport,94.48,Orient,WA,48.8878,-118.2105,149,Special educational needs teacher,6/21/1978,c9235f76ecb45e67372d79ecac2ffb37,49.513555,-117.819653,1
97,1/1/2019 6:25,Berge LLC,gas_transport,79.39,Mesa,ID,44.6255,-116.4493,129,Cartographer,12/15/1965,606b05a976ee2bca7323d6afc85e63ce,43.673602,-116.406060,0
98,1/1/2019 6:25,Pagac LLC,shopping_pos,5.05,Powell Butte,OR,44.2415,-121.0113,2016,Local government officer,12/25/1953,d44f496e9daab912914ac9513b038df2,44.245679,-121.333658,1


In [62]:
df.columns

Index(['trans_date_trans_time', 'merchant', 'category', 'amt', 'city', 'state',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [63]:
df.is_fraud.value_counts()

is_fraud
1    75
0    25
Name: count, dtype: int64

## Dropping irrelevant columns

In [65]:
df1 = df.drop (['merchant', 'category', 'city', 'state', 'job', 'trans_num', 'dob', 'trans_date_trans_time', 'long', 'merch_long'], axis =1)

In [66]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   amt        100 non-null    float64
 1   lat        100 non-null    float64
 2   city_pop   100 non-null    int64  
 3   merch_lat  100 non-null    float64
 4   is_fraud   100 non-null    int64  
dtypes: float64(3), int64(2)
memory usage: 4.0 KB


In [67]:
df1.info

<bound method DataFrame.info of        amt      lat  city_pop  merch_lat  is_fraud
0   107.23  48.8878       149  49.159047         1
1   220.11  42.1808      4154  43.150704         1
2    96.29  41.6125       589  41.657520         1
3     7.77  32.9396       899  32.863258         1
4     6.85  43.0172       471  43.753735         0
..     ...      ...       ...        ...       ...
95  199.55  39.5994    320420  38.619691         1
96   94.48  48.8878       149  49.513555         1
97   79.39  44.6255       129  43.673602         0
98    5.05  44.2415      2016  44.245679         1
99   69.72  39.7417       271  39.053407         0

[100 rows x 5 columns]>

In [68]:
df1= df1.astype(float)

In [69]:
df1

,amt,lat,city_pop,merch_lat,is_fraud
0,107.23,48.8878,149.0,49.159047,1.0
1,220.11,42.1808,4154.0,43.150704,1.0
2,96.29,41.6125,589.0,41.657520,1.0
3,7.77,32.9396,899.0,32.863258,1.0
4,6.85,43.0172,471.0,43.753735,0.0
...,...,...,...,...,...
95,199.55,39.5994,320420.0,38.619691,1.0
96,94.48,48.8878,149.0,49.513555,1.0
97,79.39,44.6255,129.0,43.673602,0.0
98,5.05,44.2415,2016.0,44.245679,1.0


In [41]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
numeric_data = df[numeric_columns]


In [42]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)

In [43]:
scaled_df1 = pd.DataFrame(scaled_data, columns=numeric_columns)

## Splitting the data

In [70]:
# get predictors and the target variable
X = df1.drop(columns=['is_fraud'])  
y = df1['is_fraud'] 

In [71]:
#importing the standant scaler module 
from sklearn.preprocessing import StandardScaler

###  importing train_test_split

In [72]:
# importing the train test split module to split the dataset into test and train
from sklearn.model_selection import train_test_split

In [73]:
X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)

### LinearRegression model

In [74]:
from sklearn.linear_model import LinearRegression

In [75]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [76]:
print (reg.score(X_train, y_train))

0.046116168127615875


### LinearRegression

In [77]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

RandomForestRegressor()

In [78]:
print(forest_reg.score(X_test,y_test))

-1.2799134615384609


###  DecisionTreeRegressor

In [79]:
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor()
DTR.fit(X_train,y_train)

DecisionTreeRegressor()

In [80]:
print (DTR.score(X_train, y_train))

1.0


### DecisionTreeClassifier

In [81]:
from sklearn.tree import DecisionTreeClassifier
DTF = DecisionTreeClassifier()
DTF.fit(X_train, y_train)

DecisionTreeClassifier()

In [82]:
print (DTF.score(X_train, y_train))

1.0


### RandomForestClassifier

In [83]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

RandomForestClassifier()

In [84]:
print (RFC.score(X_train, y_train))

1.0


### Result Simulation 

In [88]:
input=(70, 40,300, 50)
input_as_numpy=np.asarray(input)
input_reshaped=input_as_numpy.reshape(1,-1)
pre1=RFC.predict(input_reshaped)
if(pre1==1): 
  print("Happy customer:(")
else:
  print("Sad customer:)")

Happy customer:(


C:\Users\ADDIS\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [89]:
input=(200,150,89,50)
input_as_numpy=np.asarray(input)
input_reshaped=input_as_numpy.reshape(1,-1)
pre1=RFC.predict(input_reshaped)
if(pre1==1): 
  print("Happy customer:(")
else:
  print("Sad customer:)")

Sad customer:)


C:\Users\ADDIS\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
